In [1]:
# 
# notebook setup
#
try:
    %matplotlib qt
    import matplotlib.pyplot as plt
    #
    # Temporary hack needed to make the gridtools package visible from the notebook.
    # No longer needed when GT4Py will be installed as a regular Python package (i.e. through  setup.py)
    #
    import os
    os.chdir(os.path.abspath('..'))
except Exception:
    print ("### WARNING: plotting not available")
else:
    import numpy as np
    print ("Plotting enabled")

Plotting enabled


# ::: Gridtools4Py :::
## A Python interface for Gridtools

### A copy stencil implemented in Python


In [2]:
from gridtools.stencil import MultiStageStencil, stencil_kernel


class CopyStencil (MultiStageStencil):
    """
    Definition of a simple copy stencil.-
    """
    @stencil_kernel
    def kernel (self, out_data, in_data):
        """
        The entry stage of this stencil.-
        """
        #
        # iterate over the interior data points
        #
        for p in self.get_interior_points (out_data):
            out_data[p] = in_data[p]


### Use NumPy arrays as data fields

In [3]:
domain = (64, 64, 32)

source = np.random.rand (*domain)   # data field of size 'domain'
                                    # filled with random numbers
target = np.zeros (domain)          # data field of size 'domain'
                                    # filled with zeros

### Run the stencil in Python mode

In [4]:
copy         = CopyStencil ( )      # instance of the stencil defined above
copy.backend = "python"             # will run in Python only mode

%timeit -n1 -p3 -r 10 copy.run (in_data=source, out_data=target) # execute it

1 loops, best of 10: 68.2 ms per loop


### Run the *same* stencil in C++ mode

In [5]:
copy.backend = "c++"        # will run using Gridtools in C++

%timeit -n1 -p3 -r 10 copy.run (in_data=source, out_data=target) # execute it

1 loops, best of 10: 122 µs per loop


### The code has been translated, compiled and dynamically linked into the current session

In [6]:
copy.compiler.lib_handle

<CDLL '/tmp/__gridtools_mup3xxi5/libgridtools4py.0001.so', handle 25263e0 at 7fd2df4fe080>

### Example: the Laplace operator

In [7]:
class Laplace (MultiStageStencil):
    @stencil_kernel
    def kernel (self, out_data, in_data):
        """
        The user must always define a 'kernel' function.-
        """
        for p in self.get_interior_points (out_data):
            out_data[p] = -4.0 * in_data[p] - (
                          in_data[p + (1,0,0)] + in_data[p + (0,1,0)] + 
                          in_data[p + (-1,0,0)] + in_data[p + (0,-1,0)])

### Run it in Python, C++ and CUDA modes

In [8]:
lap = Laplace ( )
lap.set_halo        ( (1, 1, 1, 1) )
lap.set_k_direction ("forward")

lap.backend = "python"
%timeit -n 1 -p 3 -r 5 lap.run (in_data=source, out_data=target)

lap.backend = "cuda"
%timeit -n 1 -p 3 -r 5 lap.run (in_data=source, out_data=target)

lap.backend = "c++"
%timeit -n 1 -p 3 -r 5 lap.run (in_data=source, out_data=target)

1 loops, best of 5: 635 ms per loop
1 loops, best of 5: 44.7 µs per loop
1 loops, best of 5: 243 µs per loop


### Stencil combination example: horizontal diffusion

In [9]:
class FluxI (MultiStageStencil):
    @stencil_kernel
    def kernel (self, out_fli, in_lapi):
        for p in self.get_interior_points (out_fli):
            out_fli[p] = in_lapi[p + (1,0,0)] - in_lapi[p]

class FluxJ (MultiStageStencil):
    @stencil_kernel
    def kernel (self, out_flj, in_lapj):
        for p in self.get_interior_points (out_flj):
            out_flj[p] = in_lapj[p + (0,1,0)] - in_lapj[p]

class Out (MultiStageStencil):
    @stencil_kernel
    def kernel (self, out_hr, in_wgt, in_fli, in_flj):
        for p in self.get_interior_points (out_hr):
            out_hr[p] = in_wgt[p] * ( in_fli[p + (-1,0,0)] - in_fli[p] +
                                      in_flj[p + (0,-1,0)] - in_flj[p] )            

In [10]:
#
# combine the four stencils together
#
lap = Laplace ( )
lap.set_halo ( (2,2,2,2) )

fli = FluxI ( )
fli.set_halo ( (1,1,1,1) )

flj = FluxJ ( )
flj.set_halo ( (1,1,1,1) )

out = Out ( )
out.set_halo ( (0,0,0,0) )

In [11]:
#
# given a restriction of unique parameter names,
# we may build a combined stencil like this
#
# - CURRENTLY NOT WORKING -
#
#hor_dif = out.build (output='out_hr',
#                     in_fli=fli.build (output='out_fli',
#                                       in_lapi=lap.build (output='out_data')),
#                     in_flj=flj.build (output='out_flj',
#                                       in_lapj=lap.build (output='out_data')))

### Stencil combination example: execution and data-dependency graphs

In [12]:
#hor_dif.plot_execution_graph ( )
#hor_dif.plot_data_graph ( )

In [13]:
#
# - CURRENTLY NOT WORKING -
# ``hor_dif`` is now an independent stencil and can run in C++ mode
#

#hor_dif.backend = "c++"
#hor_dif.run (out_hr = target,
#             in_wgt = wgt,
#             in_data= source)

### Plotting

In [14]:
from tests.test_stencils import HorizontalDiffusion

#
# initialize the input data
#
for i in range (domain[0]):
    for j in range (domain[1]):
        for k in range (domain[2]):
            source[i,j,k] = i**5 + j
wgt    = np.ones (domain)
target = np.ones (domain)

hd = HorizontalDiffusion (domain)
hd.set_halo ( (2,2,2,2) )
hd.run (out_data=target,
        in_wgt=wgt,
        in_data=source)

In [15]:
hd.plot_3d (source[:,:,0])
hd.plot_3d (target[2:-2,2:-2,0])

### Stencil stages as function - independent stages

In [16]:
class ShallowWater (MultiStageStencil):
    def __init__ (self, domain):
        super ( ).__init__ ( )
        #
        # constants to callibrate the system
        #
        self.bl     = 0.2
        self.growth = 1.2
        self.dt     = 0.05

        #
        # temporary data fields
        #
        self.Hd = np.zeros (domain)
        self.Ud = np.zeros (domain)
        self.Vd = np.zeros (domain)
        self.Hx = np.zeros (domain)
        self.Ux = np.zeros (domain)
        self.Vx = np.zeros (domain)
        self.Hy = np.zeros (domain)
        self.Uy = np.zeros (domain)
        self.Vy = np.zeros (domain)

        self.L  = np.zeros (domain)
        self.R  = np.zeros (domain)
        self.T  = np.zeros (domain)
        self.B  = np.zeros (domain)

        self.Dh = np.zeros (domain)
        self.Du = np.zeros (domain)
        self.Dv = np.zeros (domain)
        
        
    def stage_momentum (self, out_M, out_Md, out_Mx, out_My):
        for p in self.get_interior_points (out_M):
            self.L[p] = out_M[p + (-1,0,0)]
            self.R[p] = out_M[p + (1,0,0)]
            self.T[p] = out_M[p + (0,1,0)]
            self.B[p] = out_M[p + (0,-1,0)]

            out_Mx[p] = self.R[p] - self.L[p]
            out_My[p] = self.T[p] - self.B[p]

            out_Md[p] = out_M[p] * (1.0 - self.bl) + self.bl * (0.25 * (self.L[p] + 
                                                                        self.R[p] +
                                                                        self.T[p] +
                                                                        self.B[p]))

    def stage_dynamics (self, out_H, in_Hd, in_Hx, in_Hy,
                              out_U, in_Ud, in_Ux, in_Uy,
                              out_V, in_Vd, in_Vx, in_Vy):
        for p in self.get_interior_points (out_H):
            self.Dh[p] = -in_Ud[p] * in_Hx[p] -in_Vd[p] * in_Hy[p] - in_Hd[p] * (in_Ux[p] + in_Vy[p])
            self.Du[p] = -in_Ud[p] * in_Ux[p] -in_Vd[p] * in_Uy[p] - self.growth * in_Hx[p]
            self.Dv[p] = -in_Ud[p] * in_Vx[p] -in_Vd[p] * in_Vy[p] - self.growth * in_Hy[p]
            #
            # take first-order Euler step
            #
            out_H[p] = in_Hd[p] + self.dt * self.Dh[p];
            out_U[p] = in_Ud[p] + self.dt * self.Du[p];
            out_V[p] = in_Vd[p] + self.dt * self.Dv[p];


    @stencil_kernel
    def kernel (self, out_H, out_U, out_V):
        #
        # momentum calculation for each field
        #
        self.stage_momentum (out_M  = out_U,
                             out_Md = self.Ud,
                             out_Mx = self.Ux,
                             out_My = self.Uy)
        self.stage_momentum (out_M  = out_V,
                             out_Md = self.Vd,
                             out_Mx = self.Vx,
                             out_My = self.Vy)
        self.stage_momentum (out_M  = out_H,
                             out_Md = self.Hd,
                             out_Mx = self.Hx,
                             out_My = self.Hy)
        #
        # dynamics and momentum combined
        #
        for p in self.get_interior_points (out_H):
            self.Dh[p] = -self.Ud[p] * self.Hx[p] -self.Vd[p] * self.Hy[p] - self.Hd[p]  * (self.Ux[p] + self.Vy[p])
            self.Du[p] = -self.Ud[p] * self.Ux[p] -self.Vd[p] * self.Uy[p] - self.growth * self.Hx[p]
            self.Dv[p] = -self.Ud[p] * self.Vx[p] -self.Vd[p] * self.Vy[p] - self.growth * self.Hy[p]
            #
            # take first-order Euler step
            #
            out_H[p] = self.Hd[p] + self.dt * self.Dh[p]
            out_U[p] = self.Ud[p] + self.dt * self.Du[p]
            out_V[p] = self.Vd[p] + self.dt * self.Dv[p]

/home/docker/venv/lib/python3.4/site-packages/matplotlib/collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):


### Stage execution path and data-dependency graph

In [17]:
sw = ShallowWater (domain)
sw.set_halo ( (1,1,1,1) )
sw.backend = 'c++'
sw.run (out_H = target,
        out_U = source,
        out_V = wgt)

In [18]:
sw.plot_data_dependency (sw.scope.stage_execution)
for stg in sw.scope.stage_execution:
    print (stg.name)

shallowwater_007_stage_momentum_000
shallowwater_007_stage_003
shallowwater_007_stage_momentum_002
shallowwater_007_stage_momentum_001


In [20]:
sw.plot_data_dependency ( )

### Animations

In [21]:
from tests.test_sw import SWTest

anim = SWTest ( )
anim.setUp ( )
anim.test_animation (50000)

## : Game of Life :

In [22]:
class GameOfLife (MultiStageStencil):
    def __init__ (self, domain):
        super ( ).__init__ ( )
        self.counter = np.zeros (domain)
        
    @stencil_kernel
    def kernel (self, out_X):
        for p in self.get_interior_points (out_X):
            self.counter[p] = out_X[p + (1,0,0)]  + out_X[p + (1,1,0)]   + \
                              out_X[p + (0,1,0)]  + out_X[p + (-1,1,0)]  + \
                              out_X[p + (-1,0,0)] + out_X[p + (-1,-1,0)] + \
                              out_X[p + (0,-1,0)] + out_X[p + (1,-1,0)]
            if out_X[p] == 1.0 and self.counter[p] == 2:
                out_X[p] = 0.0
            elif self.counter[p] == 3:
                out_X[p] = 1.0
            else:
                out_X[p] = 0.0

In [23]:
X = [ [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
      [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
      [0,0,1,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],     
      [0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],     
      [0,0,1,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
      [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],     
      [0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0],
      [0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0],
      [0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0],
      [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
      [0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0],
      [1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1],
      [1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1],
      [1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1],
      [0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,1,0,1,1,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0],
      [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
      [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
      [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
      [0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0],
      [1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1],
      [1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,1,0,1,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1],
      [1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1],
      [0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0],
      [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
      [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
      [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
      [0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,1,0,1,1,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0],
      [1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1],
      [1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1],
      [1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1],
      [0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0],
      [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
      [0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0],
      [0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0],
      [0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0],
      [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
      [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
      [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
      [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],     
      [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],     
      [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
      [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0] ]

X = np.array (X).reshape (42, 42, 1)

In [24]:
X = np.array (np.random.randint (2,
                                 size=X.shape),
              dtype=np.float64)
gol = GameOfLife (X.shape)
gol.backend = 'c++'
gol.set_halo ( (1,1,1,1) )

### Create a Matplotlib 2D animation

In [25]:
import matplotlib.animation as animation

fig = plt.figure ( )
im  = plt.imshow (X[:,:,0])

def anim_init ( ):
    im.set_data (X[:,:,0])
    return [im]

def anim_frame (i):
    gol.run (out_X = X)
    im.set_data (X[:,:,0])
    return [im]

ERROR:root:Error while compiling 'Sw_009'
ERROR:root:Error while compiling 'Sw_009'
ERROR:root:Error while compiling 'Sw_009'
ERROR:root:Error while compiling 'Sw_009'


In [26]:
anim = animation.FuncAnimation (fig,
                                anim_frame,
                                init_func=anim_init,
                                frames=200,
                                interval=100,
                                blit=True)

In [27]:
del anim

ERROR:root:Error while compiling 'Sw_009'


### Performance comparison (pure C++ <--> Python with C++ backend) 

```
* Copy                 11342 FPS <--> 6710 FPS (~39% overhead)
* Laplace               4276 FPS <--> 3732 FPS (~13% overhead)
* Horizontal diffusion   633 FPS <-->  550 FPS (~13% overhead)
* Shallow water         2710 FPS <--> 2525 FPS (~7% overhead)
```

#### The constant overhead is on the Python side due to library and parameter handling

## : Status update :

### Done as of 18. Aug 2015

* copy, laplace, double laplace, horizontal diffusion, shallow water stencils and game of life on CPU/GPU;
* automatic detection of read-only fields based on source analysis;
* automatic construction and display of execution and data dependency graphs by means of source analysis;
* stencil definition containing "independent" stages, which may be potentially executed in parallel;
* stage definition as a function, which avoids code replication and enables per-stage user settings;
* integrated 3d plotting capabilities;
* integrated 3d animations using an OpenGL backend;
* tutorial IPython Notebook;
* integration with CMake and the C++ testing infrastructure (Jenkins);
* integrate the testing infrastructure on CSCS systems (Greina CUDA 7.0);
* solved the problem of hitting the upper limit on concurrent DLLs open;
* increased the range of constructs understood by the translator, e.g., if.-

### In progress as of 18. Aug 2015

* integration with Docker;
* definition of a stencil program as a combination of existing stencils: this approach is similar to the one presented in MODESTO (by Tobias Gysi); horizontal diffusion using this approach is working ok;
* integrate the computation/visualization infrastructure on CSCS systems (Daint with VNC and CUDA 6.5).

### To be done as of 18. Aug 2015

* connect lifetime of objects in Python to C++, e.g., tie finalize( ) with the garbage collector;
* implement boundary conditions;
* support for definition of local variables;
* support for a different offset-indexing syntax (i,j,k);
* full C++11 support.